In [1]:
from music21 import *
import math
import music21
import numpy as np
import pandas as pd
import generate_new_score
import librosa
import random

In [2]:
score = converter.parse('./bourree.xml')
# score.show('musicxml')

In [3]:
librosa.hz_to_note(444, cents=True)

'A4+16'

In [4]:
beats = score.getTimeSignatures(recurse=True)[0].numerator
print(score.getTimeSignatures()[0].ratioString)
print(score.parts[0].getElementsByClass('Measure')[0].keySignature.sharps)
measures = int(math.ceil(score.highestTime / beats)) + 1
print(measures)


4/4
1
11


In [5]:
tempo_changes = {}
for i in range(1, measures, 1):
    measure = score.parts[0].measure(i)
    for j in range(0, len(measure)):
        if isinstance(measure[j], music21.tempo.MetronomeMark):
            tempo_changes[measure.number] = measure[j].getQuarterBPM()
tempo_changes

{}

In [6]:
score.show('text')

{0.0} <music21.metadata.Metadata object at 0x1ff3ec41290>
{0.0} <music21.stream.Part Part 1>
    {0.0} <music21.instrument.Instrument 'P1: Part 1: '>
    {0.0} <music21.stream.Measure 1 offset=0.0>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.key.KeySignature of 1 sharp>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.note.Note D>
    {0.0} <music21.spanner.Slur <music21.note.Note D><music21.note.Note B>>
    {0.0} <music21.spanner.Slur <music21.note.Note B><music21.note.Note B>>
    {0.0} <music21.dynamics.Crescendo <music21.note.Note E><music21.note.Note G>>
    {0.0} <music21.spanner.Slur <music21.note.Note E><music21.note.Note D>>
    {0.0} <music21.spanner.Slur <music21.note.Note C><music21.note.Note B>>
    {0.0} <music21.spanner.Slur <music21.note.Note B><music21.note.Note G>>
    {0.0} <music21.dynamics.Crescendo <music21.note.Note B><music21.note.Note G>>
    {0.0} <music21.spanner.Slur <music21.note.Note A><music21.note.Note C#>>
    {

In [7]:
tempo_changes = {}
for i in range(1, measures, 1):
    measure = score.parts[0].measure(i)
    for j in range(0, len(measure)):
        if isinstance(measure[j], music21.tempo.MetronomeMark):
            tempo_changes[measure.number] = measure[j].getQuarterBPM()
if not tempo_changes:
    tempo_changes[1] = 120

In [8]:
durations = []
is_chord = []
for i in range(1, measures, 1):
    measure = score.parts[0].measure(i).flatten().notesAndRests
    lengths = []
    for j in range(0, len(measure)):
        s = measure[j].duration.quarterLength
        is_chord.append(False)
        lengths.append(float(s))
    durations.append(lengths)

In [9]:
measure_notes = []
measure_notes_frequency = []
for i in range(1, measures, 1):
    measure = score.parts[0].measure(i).flatten().notesAndRests
    notes = []
    notes_frequency = []
    for j in range(0, len(measure)):
        if (measure[j].isChord):
            chord_notes = []
            chord_frequencies = []
            chord = measure[j].notes
            for i in chord:
                chord_notes.append(str(i.pitch.name) + str(i.pitch.octave))
                chord_frequencies.append(i.pitch.frequency)
            # notes.append(str(chord[-1].pitch.name + str(chord[-1].pitch.octave)))
            # notes_frequency.append(chord[-1].pitch.frequency)
            notes.append(chord_notes)
            notes_frequency.append(chord_frequencies)
            continue
        elif (measure[j].isRest):
            s = 'rest'
            f = 0.0
        else:
            f = measure[j].pitch.frequency
            s = str(measure[j].pitch.name)
            s += str(measure[j].pitch.octave)
        notes.append(s)
        notes_frequency.append(f)
    measure_notes.append(notes)
    measure_notes_frequency.append(notes_frequency)

In [10]:
bpm = tempo_changes[1]
quarter_note_duration = (1 / bpm) * 60
note_duration = []
for measure in durations:
    note_duration.append([note_length * quarter_note_duration for note_length in measure])
# note_duration

In [11]:
new_durations = np.concatenate(durations, dtype=object)
def my_concat(arr: np.ndarray):
    new_arr = []
    for entry in arr:
        for note in entry:
            new_arr.append(note)
    return np.array(new_arr)
new_measure_notes = my_concat(measure_notes)
new_measure_notes_frequency = my_concat(measure_notes_frequency)
new_note_duration = np.concatenate(note_duration)

start_times = []
start_times.append(0)
curr_time = 0

for i in range(0, len(new_note_duration) - 1):
    curr_time += new_note_duration[i]
    start_times.append(curr_time)

note_type = [duration.Duration(quarterLength=quarter_note_length).type for quarter_note_length in new_durations]

In [12]:

df = pd.DataFrame({'Note Type': note_type, 'Duration': new_note_duration, 
                   'Note Name': new_measure_notes, 'Note Frequency': new_measure_notes_frequency, 
                   'Start Time': start_times})

In [13]:
df['Duration'][2] = 0.8
df['Duration'][7] = 0.05
df['Duration'][12] = 0.1

C:\Users\antho\AppData\Local\Temp\ipykernel_17936\1089685760.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Duration'][2] = 0.8
C:\Users\antho\AppData\Local\Temp\ipykernel_17936\1089685760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Duration'][7] = 0.05
C:\Users\antho\AppData\Local\Temp\ipykernel_17936\1089685760.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Duration'][12] = 0.1


In [14]:
df.insert(4, "Cents", [random.randint(-10, 10) for i in range(len(df))])

In [15]:
df

,Note Type,Duration,Note Name,Note Frequency,Cents,Start Time
0,quarter,0.500,D5,587.329536,10,0.000
1,quarter,0.500,D5,587.329536,2,0.500
2,quarter,0.800,B4,493.883301,8,1.000
3,eighth,0.250,C5,523.251131,2,1.500
4,eighth,0.250,B4,493.883301,-2,1.750
5,eighth,0.250,A4,440.000000,-6,2.000
6,eighth,0.250,G4,391.995436,9,2.250
7,quarter,0.050,E5,659.255114,6,2.500
8,half,1.000,G5,783.990872,-2,3.000
9,eighth,0.250,F#5,739.988845,5,4.000


In [16]:
test = generate_new_score.AudioAnalysis(score='./bourree.xml', input_df=df)
test.generate_overlay_score()
print(test.compare_dataframe())

Correct
Correct
Long
Correct
Correct
Correct
Correct
Short
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
Correct
   Note Type  Duration Note Name Played Notes Note Status  Input Duration  \
0    quarter     0.500        D5           D5       Sharp           0.500   
1    quarter     0.500        D5           D5     Correct           0.500   
2    quarter     0.500        B4           B4       Sharp           0.800   
3     eighth     0.250        C5           C5     Correct           0.250   
4     eighth     0.250        B4           B4     Correct           0.250   
5     eighth     0.250        A4           A4        Flat           0.250   
6     eighth     0.250        G4           G4       Sharp         